# Test to get "/ListSensors" data

In [1]:
from breathe_london_api_list_sensors import main, logger
import os
import logging

logging.getLogger().addHandler(logging.StreamHandler())
from dotenv import load_dotenv

load_dotenv()

df, site_codes, output_file = main(
    api_key=os.getenv("API_KEY"), output_file="my_output.csv", site_code_limit=10
)

if df is None:
    logger.error("Failed to fetch data. Please check your API key and try again later.")
else:
    logger.info(f"Data saved to: {output_file}")
    logger.info(f"First few site codes: {site_codes[:5]}")

2024-09-03 21:47:08,741 - INFO - Starting main process
Starting main process
2024-09-03 21:47:08,742 - INFO - Using provided API key
Using provided API key
2024-09-03 21:47:08,742 - INFO - Initiating API request to fetch sensor data
Initiating API request to fetch sensor data
2024-09-03 21:47:08,742 - INFO - Attempt 1 to fetch data
Attempt 1 to fetch data
2024-09-03 21:47:18,818 - WARNING - Request timed out. Retrying... (Elapsed time: 10.08s)
Request timed out. Retrying... (Elapsed time: 10.08s)
2024-09-03 21:47:19,826 - INFO - Attempt 2 to fetch data
Attempt 2 to fetch data
2024-09-03 21:47:29,931 - WARNING - Request timed out. Retrying... (Elapsed time: 21.19s)
Request timed out. Retrying... (Elapsed time: 21.19s)
2024-09-03 21:47:30,939 - INFO - Attempt 3 to fetch data
Attempt 3 to fetch data
2024-09-03 21:47:41,008 - ERROR - API request timed out after 30 seconds
API request timed out after 30 seconds
2024-09-03 21:47:41,025 - ERROR - Your BreatheLondon API Limit exceeded
Your Bre

# Test to hourly "/getClarityData" data

In [ ]:
import requests
import os
import pandas as pd
from datetime import datetime, timedelta
import pandas as pd
import time
import seaborn as sns
import matplotlib.pyplot as plt
from dotenv import load_dotenv
load_dotenv()
API_KEY = os.getenv('API_KEY')  # Replace with your actual API key

pd.set_option('display.max_columns', None)
pd.options.mode.chained_assignment = None  # default='warn'

today = pd.Timestamp.today().strftime('%Y-%m-%d-%H-%M-%S')

API_CLARITY_HOURLY_URL = "https://api.breathelondon.org/api/getClarityData/{siteCode}/{species}/{startTime}/{endTime}/{averaging}?key={apiKey}"

def get_clarity_data(siteCode, species, startTime, endTime, averaging):
    try:
    # Format the API URL with required parameters
        formatted_startTime = startTime.replace(" ", "%20")
        formatted_endTime = endTime.replace(" ", "%20")
        url = API_CLARITY_HOURLY_URL.format(siteCode=siteCode, species=species, startTime=formatted_startTime, endTime=formatted_endTime, averaging=averaging, apiKey=API_KEY)
    
        response = requests.get(url)
    
        # Check if request was successful
        if response.status_code == 200:
            sensors = response.json()
            sensors = pd.DataFrame.from_dict(sensors)
            return sensors
        else:
            print(f"Error with status code: {response.status_code}")
            print(response.text)
            return None
    
    except ValueError:
        print("Received an unexpected response:")
        print(response.text)
        return None

# Add sitecodes needed here
siteCodes = ["CLDP0001", "CLDP0002"]  
species1 = "IPM25"
species2 = "INO2"
averaging = "Hourly"

# Calculate start and end times
endTime = datetime.now()  # Current time
startTime = endTime - timedelta(days=365)  # One month before current time

# Convert datetime objects to strings
endTime_str = endTime.strftime("%a %d %b %Y %H:%M:%S")
startTime_str = startTime.strftime("%a %d %b %Y %H:%M:%S")

df_list = []  # List to store data for each site code

for siteCode in siteCodes:
    # Get data for first species
    data1 = get_clarity_data(siteCode, species1, startTime_str, endTime_str, averaging)
    if data1 is not None and isinstance(data1, pd.DataFrame):
        print(f"Received data for {siteCode} and {species1}")
        data1['DateTime'] = pd.to_datetime(data1['DateTime'])
        data1['mod_datetime'] = data1['DateTime'] + pd.Timedelta(hours=1)
        data1['species'] = species1
        df_list.append(data1)  # Add the DataFrame to the list
    else:
        print(f"No data received for {siteCode} and {species1}")

    # Get data for second species
    data2 = get_clarity_data(siteCode, species2, startTime_str, endTime_str, averaging)
    if data2 is not None and isinstance(data2, pd.DataFrame):
        print(f"Received data for {siteCode} and {species2}")
        data2['DateTime'] = pd.to_datetime(data2['DateTime'])
        data2['mod_datetime'] = data2['DateTime'] + pd.Timedelta(hours=1)
        data2['species'] = species2
        df_list.append(data2)  # Add the DataFrame to the list
    else:
        print(f"No data received for {siteCode} and {species2}")

# Concatenate all DataFrames in the list
if df_list:
    df_all = pd.concat(df_list)
    # print(df.head())
else:
    print("No data received for all site codes and species.")

print(df_all.mod_datetime.max()), print(df_all.SiteCode.nunique(),print(df_all.shape))
df_all.head()
